In [1]:
import sys
sys.path.append('../') 

In [4]:
from pipeline import Solar_pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
Thickness = pd.read_csv('../data/Thickness.csv')
Thickness = Thickness.drop([33]) # 9-1 drop
Thickness.reset_index(drop=True, inplace=True)
Thickness

,Thickness
0,48.266667
1,47.966667
2,48.166667
3,48.600000
4,59.533333
...,...
122,84.633333
123,98.366667
124,97.033333
125,96.266667


In [10]:
def apply_histogram_equalization(img_array):
    """Apply histogram equalization to the image."""
    img_yuv = cv2.cvtColor(img_array, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

def apply_zscore_normalization(img_array):
    """Apply Z-Score normalization to the image."""
    mean_pixel = np.mean(img_array)
    std_dev_pixel = np.std(img_array)
    normalized = (img_array - mean_pixel) / std_dev_pixel
    return normalized.astype(np.float32)

def apply_minmax_normalization(img_array):
    """Apply Min-Max normalization to the image."""
    min_pixel = np.min(img_array)
    max_pixel = np.max(img_array)
    normalized = (img_array - min_pixel) / (max_pixel - min_pixel)
    return normalized.astype(np.float32)

def apply_clahe(img_array):
    """Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the image."""
    img_yuv = cv2.cvtColor(img_array, cv2.COLOR_RGB2YUV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

def returnRGB_average(img_array):
    """Return the average of RGB values of the image."""
    return np.mean(img_array, axis=(0, 1))

img_path = '../data/box_img/'
top_img_list = [f'Top_{i}.jpg' for i in range(0, 127)]
bot_img_list = [f'Bot_{i}.jpg' for i in range(0, 127)]

top_img_histogram = []
bot_img_histogram = []

top_img_zscore = []
bot_img_zscore = []

top_img_minmax = []
bot_img_minmax = []

top_img_clahe = []
bot_img_clahe = []

for img in top_img_list:
    img_array = cv2.imread(img_path + 'Top/' + img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    #img_array = img_array.astype(np.float32)
    
    top_img_histogram.append(returnRGB_average(apply_histogram_equalization(img_array)))
    top_img_zscore.append(returnRGB_average(apply_zscore_normalization(img_array)))
    top_img_minmax.append(returnRGB_average(apply_minmax_normalization(img_array)))
    top_img_clahe.append(returnRGB_average(apply_clahe(img_array)))
    
for img in bot_img_list:
    img_array = cv2.imread(img_path + 'Bot/' + img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    
    bot_img_histogram.append(returnRGB_average(apply_histogram_equalization(img_array)))
    bot_img_zscore.append(returnRGB_average(apply_zscore_normalization(img_array)))
    bot_img_minmax.append(returnRGB_average(apply_minmax_normalization(img_array)))
    bot_img_clahe.append(returnRGB_average(apply_clahe(img_array)))
    


In [13]:
Histogram_Equalization = pd.DataFrame(
    {'Top_R': [i[0]for i in top_img_histogram], 
     'Top_G': [i[1]for i in top_img_histogram],
     'Top_B': [i[2]for i in top_img_histogram],
     'Bot_R': [i[0]for i in bot_img_histogram],
     'Bot_G': [i[1]for i in bot_img_histogram],
     'Bot_B': [i[2]for i in bot_img_histogram],
     'Thickness': Thickness['Thickness']}
)

Histogram_Equalization.to_csv('../data/Histogram_Equalization_w_B.csv', index = False)
Histogram_Equalization = Histogram_Equalization.drop(['Bot_R', 'Bot_G', 'Bot_B'], axis = 1)
Histogram_Equalization.to_csv('../data/Histogram_Equalization.csv', index = False)

Z_Score = pd.DataFrame(
    {'Top_R': [i[0]for i in top_img_zscore],
        'Top_G': [i[1]for i in top_img_zscore],
        'Top_B': [i[2]for i in top_img_zscore],
        'Bot_R': [i[0]for i in bot_img_zscore],
        'Bot_G': [i[1]for i in bot_img_zscore],
        'Bot_B': [i[2]for i in bot_img_zscore],
        'Thickness': Thickness['Thickness']}
)

Z_Score.to_csv('../data/ZScore_Normalization_w_B.csv', index = False)
Z_Score = Z_Score.drop(['Bot_R', 'Bot_G', 'Bot_B'], axis = 1)
Z_Score.to_csv('../data/ZScore_Normalization.csv', index = False)

Min_Max = pd.DataFrame(
    {'Top_R': [i[0]for i in top_img_minmax],
        'Top_G': [i[1]for i in top_img_minmax],
        'Top_B': [i[2]for i in top_img_minmax],
        'Bot_R': [i[0]for i in bot_img_minmax],
        'Bot_G': [i[1]for i in bot_img_minmax],
        'Bot_B': [i[2]for i in bot_img_minmax],
        'Thickness': Thickness['Thickness']}
)

Min_Max.to_csv('../data/MinMax_Normalization_w_B.csv', index = False)
Min_Max = Min_Max.drop(['Bot_R', 'Bot_G', 'Bot_B'], axis = 1)
Min_Max.to_csv('../data/MinMax_Normalization.csv', index = False)